# Gluconeogenesis Model Construction: Notebook

trying out just one reaction due to minspan 

In [1]:
import os
import re
import warnings
from cobra.io import *
from cobra.io.json import *
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import mass
import numpy as np
import pandas as pd
import equilibrator_api
from equilibrator_api import ComponentContribution, Q_
import sympy as sym
from cobra import Model, Reaction, Metabolite, DictList
import cobra.test
from os.path import join
from mass.util import qcqa,qcqa_model
from mass import (
    MassConfiguration, MassMetabolite, MassModel,
    MassReaction, Simulation, UnitDefinition)
from mass.io.json import save_json_model as save_json_mass_model
from mass.visualization import plot_comparison, plot_time_profile, plot_ensemble_phase_portrait, plot_ensemble_time_profile
mass_config = MassConfiguration()
mass_config.irreversible_Keq = float("inf")
print("MASSpy version: {0}".format(mass.__version__))
from six import iteritems
import matplotlib.pyplot as plt
from mass.thermo import (
    ConcSolver, sample_concentrations,
    update_model_with_concentration_solution)
import cobra_dict as c_d
import csv
import altair as alt
from minspan.minspan import minspan,nnz
# from minspan import minspan,nnz

MASSpy version: 0.1.6


In [2]:
import sys
# from minspan import minspan,nnz
print(sys.version)

3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]


In [3]:
maps_dir = os.path.abspath("maps")
data_dir = os.path.abspath("data")
dir = os.path.abspath("")
model_dir = os.path.abspath("models")
minspan_dir= os.path.abspath("minspans_csv")

In [4]:
gly_gluco_json = os.path.join(model_dir,"glycolysis_gluconeogenesis.json")
# json.save_json_model(model=trial, filename=gly_gluco_json)

In [5]:
trial=load_json_cobra_model(filename=gly_gluco_json)

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-03-01


In [6]:
# Initiate empty model
gluco = cobra.Model()

In [7]:
# add a lumped reaction to account for pyr--> pep because PYK causes no gluco minspan
reactions_from_cobra=[
    'HEX1',
 'PGI',
 'FBP',
 'PFK',
 'FBA',
 'TPI',
 'GAPD',
 'PGK',
 'PGM',
 'ENO',
#  'PEPCKm',
 'PYK',
#  'PCm',
 'LDH_L',
#  'PYRt2m',
 'ADK1',
 'ATPM',
 'NADHM',
#  'PEPtm',
#  'EX_glc__D_c',
 'G6PP']
for reaction in trial.reactions:
    if reaction.id in reactions_from_cobra:
        rxn=reaction
        gluco.add_reaction(rxn)
        print(rxn)


            


HEX1: atp_c + glc__D_c --> adp_c + g6p_c + h_c
PGI: g6p_c <=> f6p_c
FBP: fdp_c + h2o_c --> f6p_c + pi_c
PFK: atp_c + f6p_c --> adp_c + fdp_c + h_c
FBA: fdp_c <=> dhap_c + g3p_c
TPI: dhap_c <=> g3p_c
GAPD: g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
PGK: 3pg_c + atp_c <=> 13dpg_c + adp_c
PGM: 2pg_c <=> 3pg_c
ENO: 2pg_c <=> h2o_c + pep_c
PYK: adp_c + h_c + pep_c --> atp_c + pyr_c
LDH_L: lac__L_c + nad_c <=> h_c + nadh_c + pyr_c
ADK1: amp_c + atp_c <=> 2.0 adp_c
ATPM: atp_c + h2o_c --> adp_c + h_c + pi_c
NADHM: nadh_c --> h_c + nad_c
G6PP: g6p_c + h2o_c --> glc__D_c + pi_c


In [8]:
R3D=load_json_cobra_model(filename=os.path.join(model_dir,"Recon3D.json"))
metabolite_list = ['hco3_c','gtp_c', 'gdp_c','co2_c']
for met in metabolite_list:
    r3d_met= R3D.metabolites.get_by_id(met)
    gluco.add_metabolites(r3d_met)

In [9]:

pyc = {'pyr_c': -1,
'hco3_c': -1, ,'co2_c': 1,
'atp_c': -1,
#  'gtp_c': -1,'gdp_c': 1,
 'pep_c': 1,
#  'adp_c': 1,'pi_c': 1
 }
rxn = cobra.Reaction('PYC')
rxn.lower_bound=0
gluco.add_reaction(rxn)
rxn.add_metabolites(pyc)

In [10]:
gluco.metabolites.h_c.charge= 1
# trial.metabolites.h_m.charge= 1
# trial.metabolites.h_e.charge= 1
gluco.metabolites.pi_c.charge= -2
# trial.metabolites.pi_m.charge= -2

In [11]:
#Mass balance check
for r in gluco.reactions:
    print(r.id,  r.check_mass_balance())

HEX1 {}
PGI {}
FBP {}
PFK {}
FBA {}
TPI {}
GAPD {}
PGK {}
PGM {}
ENO {}
PYK {}
LDH_L {}
ADK1 {}
ATPM {}
NADHM {'charge': 2}
G6PP {}
PYC {'charge': -3, 'O': 4, 'P': 1}


In [12]:
for met in ['glc__D_c',  'h2o_c', 'h_c', 'pi_c','lac__L_c','amp_c','atp_c','adp_c',
# 'o2_c'
# 'atp_m',
'hco3_c', 'gtp_c',
'co2_c',
    # 'h_m', 'pi_m', 'adp_m', 
    'gdp_c',
    #  'co2_m' 
    ]:
    rxn = cobra.Reaction('EX_%s' % met)
    gluco.add_reaction(rxn)
    rxn.add_metabolites({met: -1})
    if met == 'glc__D_c':
        rxn.lower_bound = -1 # by convention negative exchange flux = uptake
    elif met == 'lac__L_c':
        rxn.lower_bound = -1000
    else:
        rxn.lower_bound = -1000

In [13]:
#View all the reactions that we need build
escher_builder = escher.Builder(
    model=gluco,
    map_json=os.path.join(
        maps_dir,"Gluconeogenesis.json"),highlight_missing=True)

escher_builder

Builder(highlight_missing=True)

In [14]:
gluco.reactions.EX_glc__D_c.lower_bound=-1000

In [15]:
gluco.objective = 'EX_glc__D_c'
flux_solution = gluco.optimize()
flux_solution

,fluxes,reduced_costs
HEX1,0.000000,-0.333333
PGI,-166.666667,-0.000000
FBP,166.666667,0.000000
PFK,0.000000,-0.333333
FBA,-166.666667,-0.000000
...,...,...
EX_adp_c,0.000000,-0.000000
EX_hco3_c,0.000000,-0.000000
EX_gtp_c,0.000000,-0.000000
EX_co2_c,0.000000,-0.000000


In [16]:
gluco

Name,None
Memory address,0x01d9e8347160
Number of metabolites,24
Number of reactions,29
Number of groups,0
Objective expression,1.0*EX_glc__D_c - 1.0*EX_glc__D_c_reverse_5a3d5
Compartments,c


In [17]:
#Escher FBA: shows the solutions on the map
initial_flux_data = {
    reaction: flux
    for reaction, flux in flux_solution.fluxes.items()}

#view all the reactions that we need build
escher_builder = escher.Builder(
    model=gluco,
    map_json=os.path.join(
        maps_dir,"Gluconeogenesis.json")
    , reaction_data=initial_flux_data)

# Display map in notebook
escher_builder

Builder(reaction_data={'HEX1': 0.0, 'PGI': -166.66666666666669, 'FBP': 166.66666666666669, 'PFK': 0.0, 'FBA': …

In [18]:
from cobra.io import json
import cobra.test
import os
from os.path import join
from glob import glob

In [19]:
gluco_json = os.path.join(model_dir,"gluconeogenesis_shortcut.json")
json.save_json_model(model=gluco, filename=gluco_json)

In [20]:
gluco_json

'c:\\Users\\sicil\\LiverModel\\models\\gluconeogenesis_shortcut.json'

In [21]:
for model_file in glob(gluco_json):
    model_name = model_file.split('/')[-1]
    if 'model' not in model_name:
        continue
    print(model_name)
    model= load_json_model(model_file)
    if 'NADPHM' in model.reactions:
        model.remove_reactions(['NADPHM'])
    # media = ['EX_lac__L_c', 'EX_pyr_c', 'EX_octa_c', 'EX_gln__L_c', 'EX_acetone_c', 'EX_bhb_c',
    #          'EX_glu__L_c', 'EX_ser__L_c', 'EX_cys__L_c', 'EX_gly_c', 'EX_ala__L_c', 'EX_so3_c',
    #         'EX_etoh_c', 'EX_fru_c']
    media = ['EX_glc__D_c']
    for met in media:
        if met in model.reactions:
            model.reactions.get_by_id(met).lower_bound = -1000.

    rxns = [i.id for i in model.reactions]
    # removed blocke reactions because it included ['PEPCKm', 'PCm', 'PYRt2m', 'ADK1', 'PEPtm', 'EX_h2o_c', 'EX_pi_c']
    # blocked = cobra.flux_analysis.find_blocked_reactions(model)
    # print(blocked)
    # model.remove_reactions(blocked)

    solved_fluxes = minspan(model, cores=3, verbose=False, timelimit=60)
    
    df = pd.DataFrame(solved_fluxes.copy(), index=[i.id for i in model.reactions])
    df = df/df.abs().max()
    for col in range(len(df.columns)):
        column=df.iloc[:,col]
        vals = column.values
        min_val = min([abs(i) for i in vals if i != 0])
        corr_fac=1/min_val
        # print(corr_fac)
        vals=vals*abs(corr_fac)
        df['Norm'+ str(col)]=vals
    csv_dir = os.path.join(minspan_dir,"gluco_shortcut_csv.csv")
    df.to_csv(csv_dir)
    # csv_minspan_dir = os.path.join(minspan_dir,"liver_csv_minspan.csv")
## WHere it's stored
csv_minspan_dir = os.path.join(minspan_dir,"gluco_shortcut_csv.csv")

c:\Users\sicil\LiverModel\models\gluconeogenesis_shortcut.json
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmph9rwtdx3.lp
Reading time = 0.00 seconds
: 24 rows, 58 columns, 156 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmp2cvwvv4g.lp
Reading time = 0.01 seconds
: 82 rows, 87 columns, 322 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpf8sf6_uu.lp
Reading time = 0.01 seconds
: 82 rows, 87 columns, 322 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpbocigclb.lp
Reading time = 0.01 seconds
: 82 rows, 87 columns, 322 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmp_7_uay7d.lp
Reading time = 0.01 seconds
: 82 rows, 87 columns, 322 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpvo0bhi3y.lp
Reading time = 0.01 seconds
: 82 rows, 87 columns, 322 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmph35n1

In [22]:
for model_file in glob(gluco_json):
    model_name = model_file.split('/')[-1]
    if 'model' not in model_name:
        continue
    print(model_name)
    model= load_json_model(model_file)
    if 'NADPHM' in model.reactions:
        model.remove_reactions(['NADPHM'])
    # media = ['EX_lac__L_c', 'EX_pyr_c', 'EX_octa_c', 'EX_gln__L_c', 'EX_acetone_c', 'EX_bhb_c',
    #          'EX_glu__L_c', 'EX_ser__L_c', 'EX_cys__L_c', 'EX_gly_c', 'EX_ala__L_c', 'EX_so3_c',
    #         'EX_etoh_c', 'EX_fru_c']
    media = ['EX_glc__D_c']
    for met in media:
        if met in model.reactions:
            model.reactions.get_by_id(met).lower_bound = -1000.

    rxns = [i.id for i in model.reactions]
    # removed blocke reactions because it included ['PEPCKm', 'PCm', 'PYRt2m', 'ADK1', 'PEPtm', 'EX_h2o_c', 'EX_pi_c']
    # blocked = cobra.flux_analysis.find_blocked_reactions(model)
    # print(blocked)
    # model.remove_reactions(blocked)

    solved_fluxes = minspan(model, cores=3, verbose=False, timelimit=60)
    
    df = pd.DataFrame(solved_fluxes.copy(), index=[i.id for i in model.reactions])
    df = df/df.abs().max()
    for col in range(len(df.columns)):
        column=df.iloc[:,col]
        vals = column.values
        min_val = min([abs(i) for i in vals if i != 0])
        corr_fac=1/min_val
        # print(corr_fac)
        vals=vals*abs(corr_fac)
        df['Norm'+ str(col)]=vals
    csv_dir = os.path.join(minspan_dir,"gluco_csv.csv")
    df.to_csv(csv_dir)
    # csv_minspan_dir = os.path.join(minspan_dir,"liver_csv_minspan.csv")
## WHere it's stored
csv_minspan_dir = os.path.join(minspan_dir,"gluco_csv.csv")

c:\Users\sicil\LiverModel\models\gluconeogenesis_shortcut.json
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpx5_fspdz.lp
Reading time = 0.00 seconds
: 24 rows, 58 columns, 156 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpklgydflc.lp
Reading time = 0.01 seconds
: 82 rows, 87 columns, 322 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmp104wwtj9.lp
Reading time = 0.01 seconds
: 82 rows, 87 columns, 322 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpcxpx8zbz.lp
Reading time = 0.01 seconds
: 82 rows, 87 columns, 322 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpqbx84sgb.lp
Reading time = 0.01 seconds
: 82 rows, 87 columns, 322 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmp452q823o.lp
Reading time = 0.01 seconds
: 82 rows, 87 columns, 322 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmp_07xm

In [23]:
gluco.metabolites.h_c.charge= 1
# gluco.metabolites.h_m.charge= 1
# trial.metabolites.h_e.charge= 1
gluco.metabolites.pi_c.charge= -2
# gluco.metabolites.pi_m.charge= -2

In [24]:
# making dataframe 
## WHere it's stored
csv_minspan_dir = os.path.join(minspan_dir,"gluco_csv.csv")
df = pd.read_csv(csv_minspan_dir,index_col=0) 
df

,0,1,2,3,4,5,Norm0,Norm1,Norm2,Norm3,Norm4,Norm5
HEX1,0.0,0.333333,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
PGI,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
FBP,1.0,0.666667,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0
PFK,1.0,1.000000,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0
FBA,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
TPI,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
GAPD,0.0,0.666667,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
PGK,0.0,-0.666667,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0
PGM,0.0,-0.666667,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0
ENO,0.0,0.666667,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0


In [25]:
import escher
from escher import Builder

In [26]:
initial_minspan_data = {
    reaction: flux
    for reaction, flux in df.iloc[:,5].iteritems()}

In [27]:
# New instance to prevent modifications to the existing maps
escher_builder = Builder(
    model=trial,
    map_json=os.path.join(
        maps_dir,"Gluconeogenesis.json"),
    reaction_data=initial_minspan_data)

# Display map in notebook
escher_builder

Builder(reaction_data={'HEX1': 0.0, 'PGI': 0.0, 'FBP': 0.0, 'PFK': 0.0, 'FBA': 0.0, 'TPI': 0.0, 'GAPD': 0.0, '…